In [1]:
# A.I. Assignment 5
# Learning Goals
# By the end of this lab, you should be able to:

# Get more familiar with tensors in pytorch
# Create a simple multilayer perceptron model with pytorch
# Visualise the parameters


# Task
# Build a fully connected feed forward network that adds two bits.
# Determine the a propper achitecture for this network 
# (what database you use for this problem? how many layers? how many neurons on each layer?
#  what is the activation function? what is the loss function? etc)

# Create at least 3 such networks and compare their performance 
# (how accurate they are?, how farst they are trained to get at 1 accuracy?)

# Display for the best one the weights for each layer.
import torch
import torch.nn as nn
from collections import OrderedDict
import copy
import numpy as np

In [2]:
in_channels = 2
out_channels = 2
class MLPModel(nn.Module):
    def __init__(self, function: nn.Sequential, criterion):
        super(MLPModel, self).__init__()
        self.function = function
        self.criterion = criterion
    def forward(self, x):
        return self.function(x)
    def run(self, inputs, labels, optimizer, epoch_range=1_000):
        no_of_steps = 0
        for epoch in range(epoch_range):
            no_of_steps = epoch
            optimizer.zero_grad()
            outputs = model(inputs.float())
            target = labels.float()
            loss = self.criterion(outputs, target)
            loss.backward()
            optimizer.step()
            with torch.no_grad():
                predicted = outputs.round()
                accuracy = torch.sum(torch.eq(predicted, target)) / (2 * len(target))
            if not (epoch+1) % 100:
                print(f"Criterion: {self.criterion}, "+
                        f"Loss: {loss.item()}, "+
                        f"Epoch: {epoch + 1}/{epoch_range}, "+
                        f"Accuracy: {accuracy}")
            if accuracy == 1:
                print(f"Model reached 100% accuracy at epoch {epoch+1}")
                break
        return (accuracy, no_of_steps)


In [3]:
dicts = [OrderedDict([('f11', nn.Linear(in_channels, 4)),
                     ('f12', nn.ReLU()),
                     ('f13', nn.Linear(4, out_channels))]),
        OrderedDict([('f21', nn.Linear(in_channels, 4)),
                     ('f22', nn.Tanh()),
                     ('f23', nn.Linear(4, out_channels))]),
        OrderedDict([('f31', nn.Linear(in_channels, 4)),
                     ('f32', nn.Sigmoid()),
                     ('f33', nn.Linear(4, out_channels))]),
        OrderedDict([('f11', nn.Linear(in_channels, 6)),
                     ('f12', nn.ReLU()),
                     ('f13', nn.Linear(6, out_channels))]),
        OrderedDict([('f21', nn.Linear(in_channels, 6)),
                     ('f22', nn.Tanh()),
                     ('f23', nn.Linear(6, out_channels))]),
        OrderedDict([('f31', nn.Linear(in_channels, 6)),
                     ('f32', nn.Sigmoid()),
                     ('f33', nn.Linear(6, out_channels))])]

In [4]:
inputs = torch.randint(0, 2, (1_000, 2))
labels = torch.tensor([((x+y)//2, (x+y)%2) for x,y in inputs])

In [5]:
max_crit = None
best_model = None
max_acc = float('-inf')
best_index = 0
best_eta = None
min_steps = float('inf')
models = []
index = -1
best_index = None
for layer in dicts:
    for criterion in [nn.BCEWithLogitsLoss(), nn.MSELoss(), nn.L1Loss()]:
        for eta in [0.5, 0.1, 0.01]:
            model = MLPModel(nn.Sequential(copy.deepcopy(layer)),criterion)
            index += 1
            optimizer = torch.optim.SGD(model.parameters(), lr=eta)
            models.append((model, optimizer))
            accuracy, steps = model.run(inputs, labels, optimizer)
            if accuracy > max_acc or (accuracy == 1 and steps < min_steps):
                print(f"New most accurate found at index {index}")
                max_acc = accuracy
                max_crit = criterion
                best_model = model
                best_eta = eta
                min_steps = steps
                best_index = index
            print('\n')
        print('\n')
    print('\n')
print(f"Best model was at index {best_index}, "+
      f"Criterion: {max_crit}, "+
        f"Accuracy: {max_acc}, "+
        f"Learning rate: {best_eta}, "+
         f"Steps : {min_steps}")

Criterion: BCEWithLogitsLoss(), Loss: 0.3480255901813507, Epoch: 100/1000, Accuracy: 0.26899999380111694
Criterion: BCEWithLogitsLoss(), Loss: 0.07021242380142212, Epoch: 200/1000, Accuracy: 0.0
Criterion: BCEWithLogitsLoss(), Loss: 0.035016581416130066, Epoch: 300/1000, Accuracy: 0.0
Criterion: BCEWithLogitsLoss(), Loss: 0.02244991809129715, Epoch: 400/1000, Accuracy: 0.0
Criterion: BCEWithLogitsLoss(), Loss: 0.016386086121201515, Epoch: 500/1000, Accuracy: 0.0
Criterion: BCEWithLogitsLoss(), Loss: 0.012847957201302052, Epoch: 600/1000, Accuracy: 0.0
Criterion: BCEWithLogitsLoss(), Loss: 0.010514751076698303, Epoch: 700/1000, Accuracy: 0.0
Criterion: BCEWithLogitsLoss(), Loss: 0.008900508284568787, Epoch: 800/1000, Accuracy: 0.0
Criterion: BCEWithLogitsLoss(), Loss: 0.007703145034611225, Epoch: 900/1000, Accuracy: 0.0
Criterion: BCEWithLogitsLoss(), Loss: 0.006772984284907579, Epoch: 1000/1000, Accuracy: 0.0
New most accurate found at index 0


Criterion: BCEWithLogitsLoss(), Loss: 0.

In [6]:
best = models[best_index]
print(best)
print(f"Model weights:")
for name, param in best[0].named_parameters():
    if param.requires_grad:
        print(name, param.data)

(MLPModel(
  (function): Sequential(
    (f11): Linear(in_features=2, out_features=4, bias=True)
    (f12): ReLU()
    (f13): Linear(in_features=4, out_features=2, bias=True)
  )
  (criterion): MSELoss()
), SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.5
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
))
Model weights:
function.f11.weight tensor([[ 0.2103, -0.3422],
        [-0.6209, -0.5935],
        [ 0.6203,  0.7541],
        [-0.3158, -0.2782]])
function.f11.bias tensor([-0.5531,  0.5952,  0.4952,  0.2714])
function.f13.weight tensor([[ 0.1121,  0.0716,  0.6326, -0.0404],
        [-0.1561, -0.6670, -0.1151, -0.3769]])
function.f13.bias tensor([-0.4775,  0.7059])
